## _*H2 ground state energy computation using Quantum Phase Estimation*_

This notebook demonstrates using QISKit ACQUA Chemistry to computet ground state energy of the Hydrogen (H2) molecule using QPE (Quantum Phase Estimation) algorithm. It is compared to the same energy as computed by the ExactEigensolver

This notebook populates a dictionary, that is a progammatic representation of an input file, in order to drive the qiskit_acqua_chemistry stack. Such a dictionary can be manipulated programmatically. An sibling notebook `h2_iqpe` is also provided, which showcases how the ground energies over a range of inter-atomic distances can be computed and then plotted as well.

This notebook has been written to use the PYSCF chemistry driver. See the PYSCF chemistry driver readme if you need to install the external PySCF library that this driver requires.

In [1]:
import paths
import numpy as np
import pylab
from qiskit_acqua_chemistry import ACQUAChemistry
import time

distance = 0.735
molecule = 'H .0 .0 0; H .0 .0 {}'.format(distance)

# Input dictionary to configure QISKit ACQUA Chemistry for the chemistry problem.
acqua_chemistry_qpe_dict = {
    'driver': {'name': 'PYSCF'},
    'PYSCF': {
        'atom': molecule, 
        'basis': 'sto3g'
    },
    'operator': {'name': 'hamiltonian', 'transformation': 'full', 'qubit_mapping': 'parity'},
    'algorithm': {
        'name': 'QPE',
        'num_ancillae': 9,
        'num_time_slices': 50,
        'expansion_mode': 'suzuki',
        'expansion_order': 2,
    },
    'initial_state': {'name': 'HartreeFock'},
    'backend': {
        'name': 'local_qasm_simulator',
        'shots': 100,
    }
}

acqua_chemistry_ees_dict = {
    'driver': {'name': 'PYSCF'},
    'PYSCF': {'atom': molecule, 'basis': 'sto3g'},
    'operator': {'name': 'hamiltonian', 'transformation': 'full', 'qubit_mapping': 'parity'},
    'algorithm': {
        'name': 'ExactEigensolver',
    },
}

With the two algorithms configured, we can then run them and check the results, as follows.

In [2]:
start_time = time.time()
result_qpe = ACQUAChemistry().run(acqua_chemistry_qpe_dict)
result_ees = ACQUAChemistry().run(acqua_chemistry_ees_dict)
print("--- computation completed in %s seconds ---" % (time.time() - start_time))

--- computation completed in 35.693530797958374 seconds ---


In [3]:
print('The groundtruth total ground energy is           {}.'.format(
    result_ees['energy'] - result_ees['nuclear_repulsion_energy']
))
print('The total ground energy as computed by QPE is    {}.'.format(
    result_qpe['energy'] - result_qpe['nuclear_repulsion_energy']
))
print('In comparison, the Hartree-Fock ground energy is {}.'.format(
    result_ees['hf_energy'] - result_ees['nuclear_repulsion_energy']
))


The groundtruth total ground energy is           -1.8572750302023788.
The total ground energy as computed by QPE is    -1.857136875325887.
In comparison, the Hartree-Fock ground energy is -1.8369679912029842.
